# NMF Pair (Starter)

Download the pre-processed BBC Dataset [here](http://mlg.ucd.ie/files/datasets/bbc.zip). This data consists of articles collected from 5 different categories (so, we know there should be 5 topics in our model), and the data is pre-processed.

1. Read in the terms, documents, and the term-document matrix, $M$ (this is in COOrdinate format).
2. Run Non-Negative Matrix Factorization (NMF, decomposing $M\simeq WH$) for 5 topics. Save $W$ (the term-topic matrix), and $H$ (the topic-document matrix), and look at their shapes.
3. (a) Given a topic, find the 5 terms that align most strongly with this topic. Use this function to give a name to each topic. (b) Given a document *index*, show the "weights" for the topics' contribution to information about that document (Hint: For this, you'll need $H^\top$ ... don't worry too much about what the document is, just reference the index).
4. *(Optional)* Use the documents (from #1) to determine the five category names. Given this information, and using results from #2 and #3, how well does the NMF assignment align with document category assignment?
5. *(Also Optional)* NMF provides an *approximation* for matrix decomposition. Plot the error between the actual matrix and the one derived from the NMF decomposition.

## 1. Read in Data

First, we need to download the [data](http://mlg.ucd.ie/files/datasets/bbc.zip) into a `./bbc` folder in this directory. Then, we want to get the **terms** (*bbc.terms*), the **documents** (*bbc.docs*), and then the **term-document matrix** (*bbc.mtx*). 

In [44]:
# Terms
with open('./bbc/bbc.terms') as f:
    content = f.readlines()
terms = [c.split()[0] for c in content]

# Docs
with open('./bbc/bbc.docs') as f:
    content = f.readlines()
docs = [c.split()[0] for c in content]

# Term-Doc Matrix
with open('./bbc/bbc.mtx') as f:
    content = f.readlines()

In [45]:
content[:5]

['%%MatrixMarket matrix coordinate real general\n',
 '9635 2225 286774\n',
 '1 1 1.0\n',
 '1 7 2.0\n',
 '1 11 1.0\n']

In [46]:
content.pop(0)
content.pop(0)

'9635 2225 286774\n'

In [47]:
content[:5]

['1 1 1.0\n', '1 7 2.0\n', '1 11 1.0\n', '1 14 1.0\n', '1 15 2.0\n']

This is a [**COOrdinate formatted sparse matrix**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html), where the element `'1 7 2.0\n'` implies that $M_{1,7}=2.0$, i.e., in our case, term #1 shows up in document #7 exactly 2 times. Try Googling *MatrixMarket* and this *.mtx* file extension ...

In [48]:
import numpy as np
from scipy.sparse import coo_matrix

In [103]:
from scipy.io import mmread, mmwrite
a = mmread("./bbc/bbc.mtx")

In [104]:
content = a.todense()

In [105]:
content

matrix([[1., 0., 0., ..., 0., 0., 0.],
        [5., 0., 4., ..., 0., 0., 1.],
        [2., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [100]:
len(terms)

9635

In [108]:
df = pd.DataFrame(content.T, columns=terms)

In [109]:
df

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
0,1.0,5.0,2.0,3.0,4.0,10.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2221,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2222,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Run NMF

In [90]:
from sklearn.decomposition import NMF
import pandas as pd

In [110]:
nmf = NMF(n_components=5, init='random', random_state=0)

In [111]:
doc_topic = nmf.fit_transform(df)

In [113]:
doc_topic

array([[2.10263634e-01, 0.00000000e+00, 7.27404506e-03, 0.00000000e+00,
        8.75726723e-02],
       [2.34905675e-01, 4.57278883e-02, 3.35129589e-03, 0.00000000e+00,
        0.00000000e+00],
       [1.36242496e-01, 5.55477947e-02, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       ...,
       [1.69130881e-01, 3.84345933e-01, 4.06322692e-02, 0.00000000e+00,
        2.81283950e-01],
       [7.44678036e-02, 8.84384205e-02, 5.42514984e-03, 4.32139972e-03,
        1.61927820e-01],
       [0.00000000e+00, 0.00000000e+00, 4.11532941e+00, 0.00000000e+00,
        6.64243304e-02]])

In [114]:
W = pd.DataFrame(nmf.components_.round(3))

In [121]:
W.columns = terms

In [117]:
W.shape

(5, 9635)

In [122]:
W

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
0,1.196,2.816,0.733,2.137,0.020,1.626,0.120,0.077,0.457,0.272,...,0.001,0.000,0.000,0.000,0.00,0.00,0.003,0.003,0.000,0.002
1,0.737,0.000,0.098,1.198,0.000,0.000,0.000,0.193,0.000,0.005,...,0.000,0.000,0.002,0.004,0.00,0.00,0.000,0.000,0.000,0.000
2,0.637,0.387,0.048,3.757,0.036,0.000,0.000,0.178,0.079,0.120,...,0.001,0.000,0.000,0.000,0.00,0.00,0.000,0.002,0.004,0.000
3,0.562,0.442,0.146,1.168,0.026,0.049,0.000,0.061,0.024,0.298,...,0.000,0.000,0.000,0.000,0.00,0.00,0.006,0.000,0.000,0.000
4,0.667,0.398,0.052,1.169,0.050,0.049,0.008,1.170,0.268,0.000,...,0.024,0.019,0.059,0.072,0.13,0.02,0.038,0.028,0.027,0.020


In [118]:
H = pd.DataFrame(doc_topic.round(5))

In [123]:
len(docs)

2225

In [119]:
H.shape

(2225, 5)

In [126]:
H.index = docs

In [133]:
for i in H.index:
    i = i.strip(".")

In [134]:
H

,0,1,2,3,4
business.001,0.21026,0.00000,0.00727,0.00000,0.08757
business.002,0.23491,0.04573,0.00335,0.00000,0.00000
business.003,0.13624,0.05555,0.00000,0.00000,0.00000
business.004,0.30459,0.00000,0.00000,0.00833,0.00000
business.005,0.10993,0.00095,0.00950,0.00647,0.03807
...,...,...,...,...,...
tech.397,0.07824,0.00000,0.00000,0.00000,0.39175
tech.398,0.04501,0.03267,0.00000,0.00000,0.30694
tech.399,0.16913,0.38435,0.04063,0.00000,0.28128
tech.400,0.07447,0.08844,0.00543,0.00432,0.16193


## 3. Top Topic Terms

In [10]:
import numpy as np

# You may need to use `.argsort()` and the transpose function `.T` for NumPy arrays

In [128]:
np.argsort(W)

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
0,7183,7797,7796,7795,7794,7793,7792,7791,7799,6208,...,535,128,243,174,4668,39,47,1998,57,212
1,4817,5363,5360,5359,5358,5357,5355,5353,5352,5350,...,665,866,193,7666,846,3096,604,174,1406,1238
2,4817,4522,4521,4520,4517,4516,4515,4513,4512,4511,...,16,878,623,1035,600,460,811,3,553,1368
3,9634,4132,4129,4125,4122,4119,4118,4116,4115,4112,...,362,5733,5666,1136,83,1812,212,150,4634,864
4,3577,7691,7692,7693,4170,4169,7694,4164,4163,4162,...,16,2646,35,2096,150,60,1096,1279,1230,846


## 4. Document Classification

In [17]:
categories = [d[:d.find('.')] for d in docs]

In [25]:
from sklearn.metrics import classification_report

## 5. NMF Is An Approximation

It's important to remember that the `sklearn.decomposition.NMF` implementation is an approximation. That is, it is generally not the case that $M = WH$, strictly because of the constraints that we put on $W$ and $H$ (namely that they are non-negative, and sometimes $H$ should be orthonormal), so we accept some level of error. That is

$$
M \simeq WH\quad \text{or,}\quad M = WH + U
$$

where $U$ is some residual.

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('whitegrid')

Given the above discussion, let's see what $\hat{M} = WH$ actually looks like in comparison to $M$.